# data and environment 

In [1]:
import os 
import time

from tensorflow import keras
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV

In [48]:
data = keras.datasets.mnist.load_data()

In [50]:
X_train, X_val, y_train, y_val = train_test_split(data[0][0], data[0][1])
X_test, y_test = data[1][0], data[1][1]

In [56]:
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

# Implementation of basic model

In [19]:
def build_model(lr_choice=0.001):
    
    model = keras.models.Sequential([
            keras.layers.Flatten(input_shape=[28,28]),
            keras.layers.Dense(units=60, activation=keras.activations.relu),
            keras.layers.Dense(units=60, activation=keras.activations.relu),
            keras.layers.Dense(units=10, activation=keras.activations.softmax)])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_choice),
                  loss=keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])
    
    return model

### TF way

In [81]:
model = build_model()

model.fit(X_train, y_train,
          epochs=25, 
          validation_data=(X_val, y_val),
          callbacks=[keras.callbacks.EarlyStopping(patience=10),],
          verbose=0)

model.evaluate(X_test, y_test)

Epoch 1/25
1407/1407 - 3s - loss: 1.1061 - accuracy: 0.6562 - val_loss: 67.2753 - val_accuracy: 0.6785
Epoch 2/25
1407/1407 - 2s - loss: 0.5202 - accuracy: 0.8450 - val_loss: 64.0071 - val_accuracy: 0.7312
Epoch 3/25
1407/1407 - 2s - loss: 0.4205 - accuracy: 0.8757 - val_loss: 60.5548 - val_accuracy: 0.7540
Epoch 4/25
1407/1407 - 2s - loss: 0.3798 - accuracy: 0.8886 - val_loss: 61.7314 - val_accuracy: 0.7655
Epoch 5/25
1407/1407 - 2s - loss: 0.3522 - accuracy: 0.8983 - val_loss: 55.7784 - val_accuracy: 0.7903
Epoch 6/25
1407/1407 - 2s - loss: 0.3279 - accuracy: 0.9055 - val_loss: 50.4488 - val_accuracy: 0.8044
Epoch 7/25
1407/1407 - 2s - loss: 0.3061 - accuracy: 0.9112 - val_loss: 45.1864 - val_accuracy: 0.8247
Epoch 8/25
1407/1407 - 2s - loss: 0.2871 - accuracy: 0.9163 - val_loss: 41.7207 - val_accuracy: 0.8375
Epoch 9/25
1407/1407 - 3s - loss: 0.2708 - accuracy: 0.9214 - val_loss: 37.4134 - val_accuracy: 0.8536
Epoch 10/25
1407/1407 - 3s - loss: 0.2570 - accuracy: 0.9252 - val_loss: 

[25.180408477783203, 0.9077000021934509]

In [94]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_17 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_75 (Dense)             (None, 60)                47100     
_________________________________________________________________
dense_76 (Dense)             (None, 60)                3660      
_________________________________________________________________
dense_77 (Dense)             (None, 10)                610       
Total params: 51,370
Trainable params: 51,370
Non-trainable params: 0
_________________________________________________________________


### keras way

In [20]:
keras_reg = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [21]:
path = os.path.join(os.curdir, 'TenserBoard_logs', time.strftime('hyperopt_tuning_%Y%m%d_%H%M%S'))

In [22]:
keras_reg.fit(X_train, y_train, 
              epochs=100,
              validation_data=(X_val, y_val),
              callbacks=[keras.callbacks.EarlyStopping(patience=10),
                         ],
              verbose=2)

mse_test = keras_reg.score(X_test, y_test)

Epoch 1/100
1407/1407 - 3s - loss: 2.0284 - accuracy: 0.8309 - val_loss: 0.7176 - val_accuracy: 0.8782
Epoch 2/100
1407/1407 - 2s - loss: 0.4333 - accuracy: 0.9033 - val_loss: 0.4440 - val_accuracy: 0.9011
Epoch 3/100
1407/1407 - 2s - loss: 0.3212 - accuracy: 0.9192 - val_loss: 0.3585 - val_accuracy: 0.9194
Epoch 4/100
1407/1407 - 2s - loss: 0.2621 - accuracy: 0.9310 - val_loss: 0.3774 - val_accuracy: 0.9130
Epoch 5/100
1407/1407 - 3s - loss: 0.2320 - accuracy: 0.9388 - val_loss: 0.2750 - val_accuracy: 0.9315
Epoch 6/100
1407/1407 - 2s - loss: 0.1862 - accuracy: 0.9477 - val_loss: 0.3039 - val_accuracy: 0.9265
Epoch 7/100
1407/1407 - 2s - loss: 0.1795 - accuracy: 0.9501 - val_loss: 0.2444 - val_accuracy: 0.9411
Epoch 8/100
1407/1407 - 2s - loss: 0.1532 - accuracy: 0.9565 - val_loss: 0.2195 - val_accuracy: 0.9434
Epoch 9/100
1407/1407 - 2s - loss: 0.1438 - accuracy: 0.9589 - val_loss: 0.2581 - val_accuracy: 0.9454
Epoch 10/100
1407/1407 - 2s - loss: 0.1254 - accuracy: 0.9634 - val_loss:

# GridSearch for lr value

In [24]:
params = {'learning_rate': [0.0000001, 0.000001,0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}

In [24]:
%time
rnd_search_cv = GridSearchCV(estimator=keras_reg, 
                             param_grid=params, 
                             cv=5,
                             n_jobs=-1)

In [ ]:
%time
rnd_search_cv.fit(X_train, y_train, epochs=100)

In [ ]:
import pandas as pd
pd.DataFrame(rnd_search_cv.cv_results_)

In [ ]:
rnd_search_cv.score(X_test, y_test)

# TPESuggest for best architecture

In [115]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, space_eval

In [106]:
def objective(params):
    
    print(params)
    
    model = keras.models.Sequential()
    
    model.add(keras.layers.Flatten(input_shape=[28, 28]))
              
    model.add(keras.layers.Dense(units=int(params['neuronos_first_layer']), activation=params['activation']))
    model.add(keras.layers.Dropout(params['drop_outs']))
    
    for layer in range(params['layers']):
        model.add(keras.layers.Dense(units=int(params['neurons_other_layers']), activation=params['activation']))
        model.add(keras.layers.Dropout(params['drop_outs']))
              
    model.add(keras.layers.Dense(units=10, activation=keras.activations.softmax))
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=params['learning_rate']),
                 loss=keras.losses.SparseCategoricalCrossentropy(),
                 metrics=[keras.metrics.SparseCategoricalAccuracy()])
    
    #model.summary()
    
    model.fit(X_train,
              y_train,
              batch_size=32,
              epochs=50,
              verbose=0)

    val_score = model.evaluate(X_val, y_val,
                              verbose=0)
    
    return {
        'loss':  - val_score[0],
        'accuracy': val_score[1],
        'status': STATUS_OK,
        'eval_time': time.time()
        }


In [100]:
params_test = {
    'layers' : 1,
    'neuronos_first_layer': 60,
    'neurons_other_layers': 60,
    'drop_outs': 0,
    'activation': keras.activations.relu,
    'learning_rate': 0.001,
    'epochs': 25
}

objective(params_test)

{'loss': -34.215789794921875,
 'accuracy': 0.8813333511352539,
 'status': 'ok',
 'eval_time': 1602942359.605903}

In [111]:
params = {
    'layers' : hp.choice('layers', [2, 3, 4, 5, 6]),
    'neuronos_first_layer': hp.quniform('neuronos_first_layer', 10, 500, 1),
    'neurons_other_layers': hp.quniform('neurons_other_layers', 1, 500, 1),
    'drop_outs': hp.uniform('drop_outs', 0, 1),
    'activation': hp.choice('activation', [keras.activations.relu, keras.activations.tanh]),
    'learning_rate': hp.uniform('learning_rate', 0.00001, 0.1)}

In [113]:
trials = Trials()

best = fmin(fn=objective,
           space=params,
           algo=tpe.suggest,
           max_evals=3,
           trials=trials,
           verbose=True,
           show_progressbar=True)

{'activation': <function tanh at 0x000002A22AE9F828>, 'drop_outs': 0.7445290868925796, 'layers': 5, 'learning_rate': 0.02976171135504619, 'neuronos_first_layer': 402.0, 'neurons_other_layers': 224.0}
{'activation': <function tanh at 0x000002A22AE9F828>, 'drop_outs': 0.27880638944177283, 'layers': 4, 'learning_rate': 0.014923186581504707, 'neuronos_first_layer': 200.0, 'neurons_other_layers': 203.0}
{'activation': <function tanh at 0x000002A22AE9F828>, 'drop_outs': 0.3933603852469947, 'layers': 6, 'learning_rate': 0.0657347785253221, 'neuronos_first_layer': 196.0, 'neurons_other_layers': 368.0}
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [28:13<00:00, 564.57s/trial, best loss: -7.765527725219727]


In [116]:
space_eval(params, best)

{'activation': <function tensorflow.python.keras.activations.tanh(x)>,
 'drop_outs': 0.3933603852469947,
 'layers': 6,
 'learning_rate': 0.0657347785253221,
 'neuronos_first_layer': 196.0,
 'neurons_other_layers': 368.0}